In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import scipy
from scipy.stats import norm
from math import sqrt
from scipy.optimize import minimize

In [9]:
stock = yf.Ticker("NFLX")
hist = stock.history(period="10y")
hist.drop(["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis = 1, inplace = True )
hist.rename(columns = {"Close":"S"}, inplace = True)
hist

,S
Date,
2014-06-23 00:00:00-04:00,62.788570
2014-06-24 00:00:00-04:00,62.337143
2014-06-25 00:00:00-04:00,63.458572
2014-06-26 00:00:00-04:00,62.801430
2014-06-27 00:00:00-04:00,63.154285
...,...
2024-06-14 00:00:00-04:00,669.380005
2024-06-17 00:00:00-04:00,675.830017
2024-06-18 00:00:00-04:00,685.669983


In [10]:
hist["Q"] = hist["S"]/hist["S"].shift(1)
hist.dropna(axis = 0, inplace = True)
hist

,S,Q
Date,,
2014-06-24 00:00:00-04:00,62.337143,0.992810
2014-06-25 00:00:00-04:00,63.458572,1.017990
2014-06-26 00:00:00-04:00,62.801430,0.989645
2014-06-27 00:00:00-04:00,63.154285,1.005619
2014-06-30 00:00:00-04:00,62.942856,0.996652
...,...,...
2024-06-14 00:00:00-04:00,669.380005,1.024676
2024-06-17 00:00:00-04:00,675.830017,1.009636
2024-06-18 00:00:00-04:00,685.669983,1.014560


In [11]:
class HestonModel:
    def __init__(self, hist):
        self.hist = hist
        self.arr = np.array(hist['Q'])
        self.mews = []
        self._compute_moments()

    def _compute_moments(self):
        for i in range(5):
            self.mews.append(np.mean(self.arr**(i+1)))

    def equations(self, vars):
        k, r, sigma, theta = vars
        mu1 = 1 + r
        mu2 = (r + 1)**2 + theta
        mu4 = (1 / (k * (k - 2))) * (
            k**2 * r**4 + 4 * k**2 * r**3 + 6 * k**2 * r**2 * theta - 2 * k * r**4 + 6 * k**2 * r**2 + 12 * k**2 * r * theta
            + 3 * k**2 * theta**2 - 8 * k * r**3 - 12 * k * r * theta + 4 * k**2 * r + 6 * k**2 * theta - 12 * k * r**2
            - 24 * k * r * theta - 6 * k * theta**2 - 3 * sigma**2 * theta + k**2 - 8 * k * r - 12 * k * theta - 2 * k
        )
        mu5 = (1 / (k * (k - 2))) * (
            k**2 * r**5 + 5 * k**2 * r**4 + 10 * k**2 * r**3 * theta - 2 * k * r**5 + 10 * k**2 * r**3 + 30 * k**2 * r**2 * theta
            + 15 * k**2 * r * theta**2 - 10 * k * r**4 - 20 * k * r**3 * theta + 10 * k**2 * r**2 + 30 * k**2 * r * theta + 15 * k**2 * theta**2
            - 20 * k * r**3 - 60 * k * r**2 * theta - 30 * k * r * theta**2 - 15 * sigma**2 * theta + 5 * k**2 * r + 10 * k**2 * theta
            - 20 * k * r**2 - 60 * k * r * theta - 30 * k * theta**2 - 15 * sigma**2 * theta + k**2 - 10 * k * r - 20 * k * theta - 2 * k
        )
        return mu1, mu2, mu4, mu5

    def objective(self, vars):
        k, r, sigma, theta = vars
        mu1, mu2, mu4, mu5 = self.equations(vars)
        target_mu1 = self.mews[0]
        target_mu2 = self.mews[1]
        target_mu4 = self.mews[3]
        target_mu5 = self.mews[4]
        return (mu1 - target_mu1)**2 + (mu2 - target_mu2)**2 + (mu4 - target_mu4)**2 + (mu5 - target_mu5)**2

    def calibrate_parameters(self):
        initial_guess = [1, 1, 1, 1]
        constraints = (
            {'type': 'ineq', 'fun': lambda vars: vars[2]**2},  # sigma^2 > 0
            {'type': 'ineq', 'fun': lambda vars: 2 * vars[0] * vars[3] - vars[2]**2}  # 2k*theta > sigma^2
        )

        result = minimize(self.objective, initial_guess, constraints=constraints)

        if result.success:
            self.k_opt, self.r_opt, self.sigma_opt, self.theta_opt = result.x
            print(f'Optimal values: k = {self.k_opt:.6f}, r = {self.r_opt:.6f}, sigma = {self.sigma_opt:.6f}, theta = {self.theta_opt:.6f}')
            return self.k_opt, self.r_opt, self.sigma_opt, self.theta_opt
        else:
            print('Optimization failed.')

    def c_fun(self, u, S_t, K, r, tau, kappa, theta, sigma, rho, v_t):
        M = np.sqrt((rho * sigma * 1j * u - kappa)**2 + sigma**2 * (u * 1j + u**2))
        N = (rho * sigma * 1j * u - kappa - M) / (rho * sigma * 1j * u - kappa + M)
        A = r * 1j * u * tau + (kappa * theta) / sigma**2 * (-(rho * sigma * 1j * u - kappa - M) * tau - 2 * np.log((1 - N * np.exp(-M * tau)) / (1 - N)))
        C = ((rho * sigma * 1j * u - kappa - M) * (np.exp(M*tau) - 1)) / (sigma**2 * (1 - N * np.exp(M * tau)))
        heston_cf = np.exp(A + C * v_t + 1j * u * np.log(S_t))
        return heston_cf

    def heston_call_price(self, St, K, r, tau, kappa, theta, sigma, rho, vt):
        P1 = 0
        iterations = 5000
        maxNumber = 100
        ds = maxNumber / iterations

        for j in range(1, iterations):
            s1 = ds * (2 * j + 1) / 2
            s2 = s1 - 1j
            numerator1 = self.c_fun(s2, St, K, r, tau, kappa, theta, sigma, rho, vt)
            numerator2 = K * self.c_fun(s1, St, K, r, tau, kappa, theta, sigma, rho, vt)
            denominator = np.exp(np.log(K) * 1j * s1) * 1j * s1

            P1 += ds * (numerator1 - numerator2) / denominator

        P1 = np.real(P1 / np.pi)
        P2 = P1

        call_price = St * (0.5 + P1) - K * np.exp(-r * tau) * (0.5 + P2)

        return call_price


In [12]:
heston_model = HestonModel(hist)

In [13]:
k_opt, r_opt, sigma_opt, theta_opt = heston_model.calibrate_parameters()

Optimal values: k = 1.750099, r = 0.001053, sigma = 0.006126, theta = 0.000966


In [14]:
# Calculate logarithmic returns
hist['log_ret'] = np.log(hist['S'] / hist['S'].shift(1))
# Calculate volatility as the difference between latest log return and mean

mean = hist['log_ret'].mean()       # Mean log return
latest = hist['log_ret'].iloc[-1]  # Latest log return
volatility = latest - mean

S_t = hist.iloc[-1]['S']
K = 685
r = ( ( 1 + r_opt)**252) - 1 # Adjusted for annualization
tau = 4/365
kappa = k_opt
theta = theta_opt
sigma = sigma_opt * sqrt(252)  # Adjusted for annualization
rho = 0.068
v_t = volatility

call_price = heston_model.heston_call_price(S_t, K, r, tau, kappa, theta, sigma, rho, v_t)
print(f"Heston Call Price: {call_price}")

Heston Call Price: 9.046312245166291
